In [35]:
# Import libraries
import pandas as pd
import numpy as np

In [36]:
# Load csv to a Dataframe
df = pd.read_csv('output_data\\rating_very_engaged_users.csv')

# Display number of rated movies
print(df['movieId'].nunique())

df.head(2)

25737


,userId,movieId,rating
0,54,1,4.0
1,54,2,3.0


In [37]:
# Load csv to a Dataframe
movie_df = pd.read_csv('output_data\\movie_very_engaged_users.csv')

#### Pivoting the DataFrame into a Factorization Matrix

In [38]:
# Splitting the DataFrame into 10 equal chunks to lighten Pandas work load
user_splits = np.array_split(df['userId'].unique(), 10)

# Create empty list
df_pivot_list = []

# For loop to pivot table
for split in user_splits:
    df_subset = df[df['userId'].isin(split)]
    df_pivot_subset = pd.pivot_table(df_subset, values='rating', index='userId', columns='movieId')
    df_pivot_list.append(df_pivot_subset)
    print("New split on duty !")

# Concatenate the 10 chunks
df_pivot = pd.concat(df_pivot_list)

print('Job Done: Factorization Matrix Ready')


New split on duty !
New split on duty !
New split on duty !
New split on duty !
New split on duty !
New split on duty !
New split on duty !
New split on duty !
New split on duty !
New split on duty !
Job Done: Factorization Matrix Ready


In [39]:
# Display matrix shape
df_pivot.shape

(4580, 25737)

In [40]:
# Show the first rows of the matrix
df_pivot.head(2)

movieId,1,2,3,4,5,6,7,8,9,10,...,130476,130522,130560,130604,130614,130622,130656,130828,131110,131172
userId,,,,,,,,,,,,,,,,,,,,,
54,4.0,3.0,NaN,NaN,3.0,3.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,5.0,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Recommendation with genres

In [41]:
# Load csv to a Dataframe
movie_df = pd.read_csv('output_data\\movie_blockbusters.csv')

# Set 'movieId' as the index of the DataFrame
movie_df.set_index('movieId', inplace = True)
movie_df['year'] = movie_df['year'].astype(str)
print(movie_df.shape)
movie_df.head(2)

(1377, 22)


,title,genres,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Jumanji,Adventure|Children|Fantasy,1995,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Creating Dataframe with numeric types only columns

In [42]:
movie_df_num = movie_df.select_dtypes(include=[float, int])
movie_df_num.head(2)

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,
1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


#### Create Pearson Correlation Function

In [43]:
import warnings

# Ignore warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


def matching_genres(movie_title):
    # Find index of target movie
    target_movie_index = int(movie_df.index[movie_df['title'] == movie_title][0])
    
    # Select target movie genres
    target_genres = movie_df_num.loc[target_movie_index]

    # Calculate correlation score of target movie genres
    correlations = movie_df_num.apply(lambda row: row.corr(target_genres), axis=1)
    
    # Create DataFrame with genres correlation scores
    df_genre = pd.DataFrame(correlations, columns=['PearsonG'])
    
    # Add titles, genres and year
    df_genre = df_genre.join(movie_df[['title', 'genres', 'year']])
    
    # Sort score in descending order
    df_genre = df_genre.sort_values('PearsonG', ascending=False)
    df_genre.dropna(inplace=True)

    # Calculate correlation score with movie ratings
    target = df_pivot[target_movie_index]
    similar_to_target = df_pivot.corrwith(target)
    
    # Create DataFrame with movie correlation scores
    corr_target = pd.DataFrame(similar_to_target, columns=['PearsonR'])
    
    # Drop NaN
    corr_target.dropna(inplace=True)

    # Sort score in descending order
    corr_target = corr_target.sort_values('PearsonR', ascending=False)
    
    # Cast index in int type
    corr_target.index = corr_target.index.map(int)
    
    # Join genres and ratings correlation DataFrames
    df_combined = corr_target.join(df_genre.set_index(df_genre.index), how='inner')
    
    # Sort descending PearsonR et PearsonG columns
    df_combined = df_combined.sort_values(['PearsonR', 'PearsonG'], ascending=False)
    
    # Display results
    print("Because you watch this movie :", movie_title)
    print("Our Top10 Selection of 2000's movies for you !")
    print(df_combined.head(10))#.to_string(index=False))

# Call function
#matching_genres(matching_title)


#### Call it randomly with a function to generate recommandation

In [46]:
import random

def random_movie_recommendation():
    # Randomly select a movie title from the movie_df['title'] column
    random_title = random.choice(movie_df['title'].tolist())
    
    # Print the selected title (optional)
    print(f"Randomly selected movie: '{random_title}'")
    
    # Call the matching_genres function with the randomly selected title
    matching_genres(random_title)

# Call the function to get a random movie recommendation
random_movie_recommendation()


Randomly selected movie: 'Cinderella Man'
Because you watch this movie : Cinderella Man
Our Top10 Selection of 2000's movies for you !
         PearsonR  PearsonG                  title  \
movieId                                              
33660    1.000000  1.000000         Cinderella Man   
6565     0.487816  0.687184             Seabiscuit   
1693     0.469478  0.441176                Amistad   
4995     0.465001  1.000000      Beautiful Mind, A   
4019     0.461802  0.687184      Finding Forrester   
1620     0.461586  0.243544         Kiss the Girls   
3916     0.452718  0.687184    Remember the Titans   
140      0.446767  1.000000  Up Close and Personal   
3178     0.441394  0.687184         Hurricane, The   
1096     0.426087  0.687184        Sophie's Choice   

                               genres  year  
movieId                                      
33660                   Drama|Romance  2005  
6565                            Drama  2003  
1693                    Drama|My

#### Call it manually to generate recommandation

In [45]:
matching_genres('mammuth')

IndexError: index 0 is out of bounds for axis 0 with size 0